## 重采样

In [1]:
from osgeo import gdal, osr  # 导入gdal和osr模块

In [2]:
path_img = "E:\\YNU\\5\\OpenSourceGIS\\Act_5\\OnClass\\chenggong_utm48.tif"  # 输入影像路径
path_resam = "E:\\YNU\\5\\OpenSourceGIS\\Act_5\\OnClass\\chenggong_utm48_resam.tif"  # 输出重采样影像路径
res = 60  # 重采样分辨率

#### 获取重采样前的地理转换参数

In [8]:
dset = gdal.Open(path_img)  # 打开图像文件
geo_trans = dset.GetGeoTransform()  # 获取图像的地理转换参数
print('重采样前的地理转换参数为', geo_trans)  # 打印重采样前的地理转换参数

重采样前的地理转换参数为 (268167.02525260474, 10.421454790528236, 0.0, 2765453.1872716597, 0.0, -9.543567381150961)


In [9]:
x_min, y_max = geo_trans[0], geo_trans[3] # 左上角坐标
x_max, y_min = x_min + geo_trans[1] * dset.RasterXSize, y_max + geo_trans[5] * dset.RasterYSize # 右下角坐标
print('左上角和右下角坐标', (x_min, y_min), (x_max, y_max)) # 输出左上角和右下角坐标

左上角和右下角坐标 (268167.02525260474, 2733205.4730907506) (298128.7077753734, 2765453.1872716597)


In [11]:
x_size = (x_max - x_min) / res # 计算行数
y_size = (y_max - y_min) / res # 计算列数
x_size, y_size = int(x_size), int(y_size) # 将行数和列数转换为整数
print('行数', x_size, '列数', y_size) # 输出整数的行数和列数

res_x = (x_max - x_min) / x_size # 计算x方向的分辨率
res_y = (y_max - y_min) / y_size # 计算y方向的分辨率
print('分辨率为', res_x, res_y) # 输出分辨率

行数 499 列数 537
分辨率为 60.04345194943623 60.051609275435936


In [13]:
geotrans_resample = [268167.02525260474, 60.04345194943623, 0.0, 2765453.1872716597, 0.0, -60.051609275435936] # 重采样的地理转换参数

In [14]:
driver = gdal.GetDriverByName('GTiff') # 创建数据驱动
dset_resam = driver.Create(path_resam, # 输出文件名
                           xsize = 499, # 输出列数
                           ysize = 537, # 输出行数
                           bands = dset.RasterCount, # 输出波段数
                           eType = gdal.GDT_Int16) # 输出数据类型

dset_resam.SetGeoTransform(geotrans_resample) # 设置输出文件的地理变换参数
dset_resam.SetProjection(dset.GetProjection()) # 设置输出文件的投影

resam = gdal.ReprojectImage(src_ds = dset, # 输入数据集
                            dst_ds = dset_resam, # 输出数据集
                            eResampleAlg = gdal.GRA_Bilinear # 重采样算法
                            )
dset_resam = None # 关闭输出文件